In [1]:
import os
import schnetpack as spk
from schnetpack.datasets import QM9
import schnetpack.transform as trn

import torch
import torchmetrics
import pytorch_lightning as pl

qm9tut = './qm9tut'
if not os.path.exists('qm9tut'):
    os.makedirs(qm9tut)

### Loading data

In [2]:
qm9data = QM9(
    './qm9.db',
    batch_size=100,
    num_train=1000,
    num_val=1000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        trn.RemoveOffsets(QM9.U0, remove_mean=True, remove_atomrefs=True),
        trn.CastTo32()
    ],
    property_units={QM9.U0: 'eV'},
    num_workers=1,
    split_file=os.path.join(qm9tut, "split.npz"),
    pin_memory=True, # set to false, when not using a GPU
    load_properties=[QM9.U0], #only load U0 property
)
qm9data.prepare_data()
qm9data.setup()

100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


In [3]:
atomrefs = qm9data.train_dataset.atomrefs
print('U0 of hyrogen:', atomrefs[QM9.U0][1].item(), 'eV')
print('U0 of carbon:', atomrefs[QM9.U0][6].item(), 'eV')
print('U0 of oxygen:', atomrefs[QM9.U0][8].item(), 'eV')

U0 of hyrogen: -13.613121032714844 eV
U0 of carbon: -1029.863037109375 eV
U0 of oxygen: -2042.611083984375 eV


In [4]:
means, stddevs = qm9data.get_stats(
    QM9.U0, divide_by_atoms=True, remove_atomref=True
)
print('Mean atomization energy / atom:', means.item())
print('Std. dev. atomization energy / atom:', stddevs.item())

Mean atomization energy / atom: -4.245704495720481
Std. dev. atomization energy / atom: 0.19056816253783845


In [5]:
cutoff = 5.
n_atom_basis = 30

pairwise_distance = spk.atomistic.PairwiseDistances() # calculates pairwise distances between atoms
radial_basis = spk.nn.GaussianRBF(n_rbf=20, cutoff=cutoff)
schnet = spk.representation.SchNet(
    n_atom_basis=n_atom_basis, n_interactions=3,
    radial_basis=radial_basis,
    cutoff_fn=spk.nn.CosineCutoff(cutoff)
)
pred_U0 = spk.atomistic.Atomwise(n_in=n_atom_basis, output_key=QM9.U0)

nnpot = spk.model.NeuralNetworkPotential(
    representation=schnet,
    input_modules=[pairwise_distance],
    output_modules=[pred_U0],
    postprocessors=[trn.CastTo64(), trn.AddOffsets(QM9.U0, add_mean=True, add_atomrefs=True)]
)

In [37]:
#nnpot.representation

In [7]:
output_U0 = spk.task.ModelOutput(
    name=QM9.U0,
    loss_fn=torch.nn.MSELoss(),
    loss_weight=1.,
    metrics={
        "MAE": torchmetrics.MeanAbsoluteError()
    }
)

In [8]:
task = spk.task.AtomisticTask(
    model=nnpot,
    outputs=[output_U0],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4}
)

/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [9]:
logger = pl.loggers.TensorBoardLogger(save_dir=qm9tut)
callbacks = [
    spk.train.ModelCheckpoint(
        model_path=os.path.join(qm9tut, "best_inference_model"),
        save_top_k=1,
        monitor="val_loss"
    )
]

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=qm9tut,
    max_epochs=3, # for testing, we restrict the number of epochs
)
trainer.fit(task, datamodule=qm9data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | NeuralNetworkPotential | 16.4 K
1 | outputs | ModuleList             | 0     
---------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  8.88it/s]

/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 100. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/emmahovmand/Documents/DTU/kandidat/2_semester_fall_2023/02456_Deep_learning/final_project/DeepLearningProject/.venv/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 10/10 [00:07<00:00,  1.26it/s, v_num=1, val_loss=6.700]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 10/10 [00:07<00:00,  1.26it/s, v_num=1, val_loss=6.700]


In [14]:
nnpot.representation

SchNet(
  (radial_basis): GaussianRBF()
  (cutoff_fn): CosineCutoff()
  (embedding): Embedding(100, 30, padding_idx=0)
  (interactions): ModuleList(
    (0-2): 3 x SchNetInteraction(
      (in2f): Dense(
        in_features=30, out_features=30, bias=False
        (activation): Identity()
      )
      (f2out): Sequential(
        (0): Dense(in_features=30, out_features=30, bias=True)
        (1): Dense(
          in_features=30, out_features=30, bias=True
          (activation): Identity()
        )
      )
      (filter_network): Sequential(
        (0): Dense(in_features=20, out_features=30, bias=True)
        (1): Dense(
          in_features=30, out_features=30, bias=True
          (activation): Identity()
        )
      )
    )
  )
)

In [36]:
# input
nnpot.representation.interactions[2].in2f

# output
nnpot.representation.interactions[2].f2out[1]
nnpot.representation.interactions[2].f2out[1] #.out_features

Dense(
  in_features=30, out_features=30, bias=True
  (activation): Identity()
)

In [10]:
%tensorboard --logdir=qm9tut/default

SyntaxError: cannot assign to operator (1024431672.py, line 1)

## Inference

In [39]:
import torch
import numpy as np
from ase import Atoms

best_model = torch.load(os.path.join(qm9tut, 'best_inference_model'))

In [41]:
best_model.representation.interactions[2].f2out[1].weight

Parameter containing:
tensor([[-0.1957, -0.0522,  0.1432, -0.1578, -0.0266,  0.1675, -0.1530,  0.1505,
          0.0091, -0.2785, -0.1529, -0.1676,  0.3112, -0.0173, -0.0844, -0.0365,
         -0.0081,  0.0414, -0.2603, -0.0626, -0.2589, -0.3102, -0.0970,  0.2427,
          0.2405,  0.0425,  0.2449, -0.2071, -0.1293,  0.0557],
        [-0.0068, -0.3051, -0.1168,  0.1682, -0.0147,  0.1787, -0.3157, -0.1569,
          0.1138,  0.0716, -0.0161,  0.2184,  0.2255, -0.0469,  0.0354, -0.0888,
          0.2544,  0.1859,  0.2389,  0.0581,  0.0895,  0.1545, -0.2874, -0.1504,
         -0.2749, -0.1167,  0.1257, -0.0366, -0.2447, -0.1511],
        [ 0.1583,  0.0902,  0.2954, -0.2632,  0.2900, -0.0366, -0.2473, -0.2278,
          0.0306,  0.0462, -0.1680, -0.2163,  0.2333, -0.1983,  0.0510, -0.1190,
          0.0661, -0.2612,  0.1069, -0.1161,  0.0457, -0.1743, -0.1387, -0.3095,
          0.2604,  0.1649,  0.0495, -0.1016,  0.2793, -0.2042],
        [-0.2777, -0.2464,  0.0634,  0.1861,  0.2995,  0.

In [ ]:
for batch in qm9data.test_dataloader():
    result = best_model(batch)
    print("Result dictionary:", result)
    break

Result dictionary: {'energy_U0': tensor([-11943.9487, -11506.3155, -13326.7959, -10527.4633, -12887.8292,
        -11332.6160, -12418.2834, -14844.9734, -11492.3204,  -9916.1579,
        -11819.2054, -10527.1741, -11542.2847, -12485.4260, -11944.4545,
        -12520.6219, -11368.2903, -11807.3646, -12244.1830, -12555.8003,
         -8918.0763,  -8479.8919, -11806.3710,  -9458.3451, -12556.1027,
        -10875.3029, -10390.0844, -10930.6195, -11439.7683, -10965.6649,
        -11806.4312, -10299.4354, -11438.8387,  -8917.9874, -10456.5376,
        -10563.2562, -11541.6571,  -8827.5483,  -9841.8112,  -9932.3456,
        -10930.2407, -10528.0026, -11333.0546, -11506.1835, -11404.3444,
        -12382.2623, -11470.5162, -12020.2315,  -9987.0961, -15610.5895,
        -10563.2871, -11470.8388,  -9513.3478, -11332.7022, -11979.3622,
        -11332.8466, -11380.7998,  -9931.7666, -10299.3313, -12016.0503,
        -10491.8307, -10966.1857, -11506.4515, -11368.6428, -12785.3460,
         -9915.877

In [ ]:
converter = spk.interfaces.AtomsConverter(neighbor_list=trn.ASENeighborList(cutoff=5.), dtype=torch.float32)


In [ ]:
numbers = np.array([6, 1, 1, 1, 1])
positions = np.array([[-0.0126981359, 1.0858041578, 0.0080009958],
                      [0.002150416, -0.0060313176, 0.0019761204],
                      [1.0117308433, 1.4637511618, 0.0002765748],
                      [-0.540815069, 1.4475266138, -0.8766437152],
                      [-0.5238136345, 1.4379326443, 0.9063972942]])
atoms = Atoms(numbers=numbers, positions=positions)

In [ ]:
inputs = converter(atoms)

print('Keys:', list(inputs.keys()))

pred = best_model(inputs)

print('Prediction:', pred[QM9.U0])

Keys: ['_n_atoms', '_atomic_numbers', '_positions', '_cell', '_pbc', '_idx', '_idx_i_local', '_idx_j_local', '_offsets', '_idx_m', '_idx_j', '_idx_i']
Prediction: tensor([-1103.8020], dtype=torch.float64, grad_fn=<AddBackward0>)


In [ ]:
pred.keys()

dict_keys(['energy_U0'])

In [ ]:
calculator = spk.interfaces.SpkCalculator(
    model_file=os.path.join(qm9tut, "best_inference_model"), # path to model
    neighbor_list=trn.ASENeighborList(cutoff=5.), # neighbor list
    energy_key=QM9.U0, # name of energy property in model
    energy_unit="eV", # units of energy property
    device="cpu", # device for computation
)
atoms.set_calculator(calculator)
print('Prediction:', atoms.get_total_energy())

INFO:schnetpack.interfaces.ase_interface:Loading model from ./qm9tut/best_inference_model


Prediction: -1103.8019646406174
